In [1]:
import pandas as pd

class User:
    def __init__(self):
        """ 그 유저가 어떤 영화에 평점을남겼는지 """
        self.movies = {}
    
    def add_rating(self, movie_id, rating):
        self.movies[movie_id] = rating

    def get_sim(self, other_user):
        import math
        dist = 0
        
        """ 내가 좋아하는 영화 리스트에 다른 유저도 평점을 매겼다면 이 루프안으로 들어오게 된다 """
        for movie_id, rating in self.movies.items():
            if movie_id in other_user.movies:
                # 1번 유저 rating
                # 2번 유저 other_user.movies[movie_id]
                dist += math.pow(rating - other_user.movies[movie_id], 2)
        
        # 실제 거리를 추출 하지만 이 거리를 유사도로 변환 해야 한다.
        dist = math.sqrt(dist)
        # 거리가 가까울 수록 1에 가까운 값이 나오게 된다.              
        return 1/(1+dist)  
        
        # 평점과 유사도를 곱할 것이다. 평점 데이터와 그 사람과 나의 유사도를 곱한다. 
        # 나와 유사한사람이 한 평점은 * 0.1을 해서 1이라는 결과를 가져온다.
        # 나랑 유사한 사람들이 평가한 데이터는 평점이 높게 측정이 된다.
        

def azureml_main(dataframe1 = None, dataframe2 = None):
    users = {}
    
    for index, row in dataframe2.iterrows():
        user_id = row.get('UserId')
        
        user = users.get(user_id)
        if not user:
            user = User() # 유저를 create
            users[user_id] = user 
        user.add_rating(row.get('MovieId'), row.get('Rating')) 
        
    """ 어떤유저를타겟으로 할지"""
    """ 첫번째 인자로 들어온 어떤 유저를 가져오고 싶다"""
    target_user_id = next(dataframe1.iterrows())[1].get('UserId')
    target_user = users[target_user_id]

    recommend_dict = {}
    
    """ 타켓팅하는 유저와 다른 유저들과의 거리를 구해야 한다"""
    for user_id, user in users.items():
        """ 자기 자신은 제외한다. """
        if user_id == target_user_id:
            continue
        
        """ 유사도 값을 구해온다 """    
        sim = target_user.get_sim(user)   
        
        for movie_id, rating in user.movies.items():
            # 튜플로 정의 
            # recommend[1] 평가한 사람의 수 
            recommend = recommend_dict.get(movie_id, (0, 0))
            recommend_dict[movie_id] = (
                recommend[0] + rating * sim, 
                recommend[1] + 1
                )
         
    recommend_list = []
    for movie_id, recommend in recommend_dict.items():       
        # 평점을 매기지 않은 영화들만 이 if문 안으로 들어온다.
        if movie_id not in target_user.movies:
            recommend_list.append(
                (movie_id, recommend[0] / recommend[1], recommend[1])
            )
    
    # 어떤 순서로 정렬을 할 것인지 data1, 2로 정렬할 것인데 스코어 계산 한 것, 몇명이평가했는지, 상위 10개만 출력하도록 한다.
    recommend_list = sorted(
        recommend_list,
        key=lambda data: (data[1], data[2]),
        reverse=True
    )[:10]
    
             
    return pd.DataFrame(data=recommend_list, columns=['MovieId', 'Score', 'Count']),


In [ ]:
# hybrid
import pandas as pd

def azureml_main(dataframe1 = None, dataframe2 = None):
    cloumns = next(dataframe2.iterrows())[1]
    
    dataframe2 = pd.DataFrame(
        data = [(None, columns[i], ) for i in range(1, 6)],
        cloumns = ['Movie ID', 'Movie Name', 'MovieId', 'Score']
    )
  

    return pd.concat([dataframe1, dataframe2], ignore_index = True),
